# Traitemnt OCR label par label

In [ ]:
# Etape 2 :
# Installer les bibliothèques nécessaires
!pip install easyocr
!pip install tensorflow
!pip install datasets
!pip install nltk
!pip install pandas
!pip install opencv-python-headless  # Pour éviter les conflits avec cv2

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
import cv2
import easyocr
import nltk
from PIL import Image, UnidentifiedImageError
from datasets import load_from_disk
from concurrent.futures import ThreadPoolExecutor, as_completed
import io
import pandas as pd
import time
import threading

#-------------------------------------------------------------------------------
#paramétrage de lancement
#-------------------------------------------------------------------------------
#chemin pour trouver les datasets en enregistrer le csv
results_dir = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/'
#nom du csv
csv_name ='ocr_results_label_'
#label cible sur lequel on va faire le calcul
LABEL_CIBLE = 11
#choix du datset val train ou test
type_dataset ='train'
# Nombre de threads pour le multithreading
num_threads = 24
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------

start_time = time.time()
# Create a lock
lock = threading.Lock()

# Configurer TensorFlow pour utiliser le GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU configuré pour l'utilisation.")

# Chemin du fichier CSV pour sauvegarder les résultats OCR
csv_path = results_dir +csv_name + type_dataset + str(LABEL_CIBLE)+'.csv'

# Charger le DataFrame existant si le fichier CSV existe
if os.path.exists(csv_path):
    df_ocr = pd.read_csv(csv_path)
    print(f"Chargement du fichier CSV existant : {csv_path}")
else:
    # Initialiser un DataFrame vide si le fichier n'existe pas
    df_ocr = pd.DataFrame(columns=['image_ID', 'texte_de_ocr', 'confiance_de_ocr', 'coordonnees_du_text',
                                   'coordonnees_des_zones_de_text', 'label_de_image'])
    print("Aucun fichier CSV trouvé, création d'un nouveau DataFrame.")

# Chargement des datasets
#train_dataset = load_from_disk(os.path.join(results_dir, 'train_dataset_ID_Resize'))
# test_dataset = load_from_disk(os.path.join(results_dir, 'test_dataset_ID_Resize'))
dataset = load_from_disk(os.path.join(results_dir, type_dataset+'_dataset_ID_Resize'))

# Fonction pour filtrer les images du label
def filter_images(index):
    try:
        example = dataset[index]
        if example['label'] == LABEL_CIBLE:
            return example['image'], example['label'], example['image_ID']
    except Exception as e:
        print(f"[WARNING] Erreur lors de la récupération de l'image à l'index {index}: {e}")
    return None, None, None


# Filtrer les indices pour le traitement
indices = list(range(len(dataset)))

# Utiliser ThreadPoolExecutor pour le multithreading avec une barre de progression
label_invoice_images = []
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(filter_images, i) for i in indices]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Filtrage des images", unit="image"):
        result, label, image_ID= future.result()
        if result is not None:
            label_invoice_images.append((result, label, image_ID))

print(f"Nombre d'images filtrées : {len(label_invoice_images)}")

# Fonction de prétraitement de l'image avec OpenCV
def preprocess_image(image_bytes):
    try:
        # Charger l'image avec PIL pour gérer le format .tif
        image = Image.open(io.BytesIO(image_bytes))
        image = image.convert('RGB')  # Convertir en RGB pour compatibilité OpenCV

        # Convertir l'image en un tableau NumPy
        image_np = np.array(image)

        # Redimensionnement
        #image_resized = cv2.resize(image_np, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
        image_resized = image_np

        # Conversion en niveaux de gris
        gray = cv2.cvtColor(image_resized, cv2.COLOR_RGB2GRAY)

        # Seuillage adaptatif
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        # Suppression du bruit (médian)
        blur = cv2.medianBlur(thresh, 3)

        # Encoder l'image traitée en bytes pour OCR
        _, buffer = cv2.imencode('.png', blur)
        preprocessed_image_bytes = buffer.tobytes()

        return preprocessed_image_bytes

    except Exception as e:
        print(f"Erreur de prétraitement sur l'image: {e}")
        return None

# Initialiser le lecteur EasyOCR
reader = easyocr.Reader(['en'], gpu=True)

# Fonction pour appliquer l'OCR avec EasyOCR
def apply_ocr(image_bytes):
    try:
        # Convertir les bytes en une image pour EasyOCR
        image_np = np.array(Image.open(io.BytesIO(image_bytes)))
        result = reader.readtext(image_np, detail=1)

        # Préparer les données à retourner
        texts = []
        confidences = []
        coordinates = []
        for (coord, text, confidence) in result:
            texts.append(text)
            confidences.append(confidence)
            coordinates.append(coord)

        return texts, confidences, coordinates

    except UnidentifiedImageError:
        print(f"[WARNING] Impossible d'identifier le fichier image. Ignorer...")
        return [], [], []
    except Exception as e:
        print(f"[ERROR] Erreur lors de l'application de l'OCR: {e}")
        return [], [], []

# Liste pour stocker les résultats
ocr_results = []

# Charger les noms d'image déjà traités pour éviter les doublons
images_deja_traitees = set(df_ocr['image_ID'])

# Appliquer le prétraitement et l'OCR sur toutes les images filtrées après prétraitement avec multithreading
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = {}
    for image, label, image_ID in label_invoice_images:
        # ID de l'image
        image_name = image_ID

        # Vérifier si l'image a déjà été traitée
        if image_name in images_deja_traitees:
            #print(f"Image {image_name} déjà traitée, passage...")
            continue

        # Convertir l'image en bytes directement si ce n'est pas déjà le cas
        if isinstance(image, Image.Image):
            image_bytes = io.BytesIO()
            image.save(image_bytes, format='TIFF')
            image_bytes = image_bytes.getvalue()
        else:
            image_bytes = image

        preprocessed_image_bytes = preprocess_image(image_bytes)
        if preprocessed_image_bytes:
            future = executor.submit(apply_ocr, preprocessed_image_bytes)
            futures[future] = (image_name, label, preprocessed_image_bytes)

    for future in tqdm(as_completed(futures), total=len(futures), desc="Prétraitement et OCR des images", unit="image"):
        image_name, label, preprocessed_image_bytes = futures[future]
        texts, confidences, coordinates = future.result()

        # Enregistrer les résultats dans la liste
        ocr_results.append({
            'image_ID': image_name,
            'texte_de_ocr': " ".join(texts),
            'confiance_de_ocr': confidences,
            'coordonnees_du_text': coordinates,
            'coordonnees_des_zones_de_text': [coordinates],  # Peut être la même que les coordonnées du texte dans ce contexte
            'label_de_image': label
        })

        # Sauvegarder les résultats dans le DataFrame existant et enregistrer le fichier CSV
        with lock:
          new_row = pd.DataFrame([{
              'image_ID': image_name,
              'texte_de_ocr': " ".join(texts),
              'confiance_de_ocr': confidences,
              'coordonnees_du_text': coordinates,
              'coordonnees_des_zones_de_text': [coordinates],  # Peut être la même que les coordonnées du texte dans ce contexte
              'label_de_image': label
          }])
          df_ocr = pd.concat([df_ocr, new_row], ignore_index=True)
          df_ocr.to_csv(csv_path, index=False)
          #print(f"Résultats sauvegardés pour {image_name}.")

print("Tous les résultats de l'OCR ont été enregistrés dans "+ csv_path)
end_time = time.time()
processing_time = end_time - start_time
print(f"Temps de traitement total : {round(processing_time / 60, 2)} minutes")

GPU configuré pour l'utilisation.
Aucun fichier CSV trouvé, création d'un nouveau DataFrame.


Loading dataset from disk:   0%|          | 0/53 [00:00<?, ?it/s]

Filtrage des images:   0%|          | 0/320000 [00:00<?, ?image/s]

Nombre d'images filtrées : 19947


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma